### 구글 드라이브 연결

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

### 필수 설치 라이브러리

In [5]:
# !pip install -U langchain openai

In [1]:
import os
from typing import Dict, List

from langchain.chains import ConversationChain, LLMChain, LLMRouterChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel
from langchain.chains import create_extraction_chain

from typing import Any, List, Optional
from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts import BasePromptTemplate, ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel

from langchain.chains.base import Chain
from langchain.chains.llm import LLMChain
from langchain.chains.openai_functions.utils import (
    _convert_schema,
    _resolve_schema_references,
    get_llm_kwargs,
)
from langchain.output_parsers.openai_functions import (
    JsonKeyOutputFunctionsParser,
    PydanticAttrOutputFunctionsParser,
)

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

### API 키 입력

In [2]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
#멀티프롬프트 체인의 키를 변경하기 위한 클래스
class CustomPromptChain(MultiPromptChain):
    """A custom multi-route chain based on MultiPromptChain with a modified output key."""
    @property
    def output_keys(self) -> List[str]:
        return ["out_text"]
# 추출체인의 output_key를 변경하기 위한 클래스
llm = ChatOpenAI(temperature=0.1, max_tokens=400, model="gpt-3.5-turbo")
class CustomExtractionChain(LLMChain):
    """A custom extraction chain based on LLMChain with a modified output key."""
    output_key: str = "extracted_data"

def _get_extraction_function(entity_schema: dict) -> dict:
    return {
        "name": "information_extraction",
        "description": "Extracts the relevant information from the passage.",
        "parameters": {
            "type": "object",
            "properties": {
                "info": {"type": "array", "items": _convert_schema(entity_schema)}
            },
            "required": ["info"],
        },
    }
_EXTRACTION_TEMPLATE = """Extract and save the relevant entities mentioned \
in the following passage together with their properties.

Only extract the properties mentioned in the 'information_extraction' function.

If a property is not present and is not required in the function parameters, do not include it in the output.

Passage:
{input}
"""  # noqa: E501


def create_custom_extraction_chain(
    schema: dict,
    llm: BaseLanguageModel,
    prompt: Optional[BasePromptTemplate] = None,
    tags: Optional[List[str]] = None,
    verbose: bool = False,
) -> Chain:

    function = _get_extraction_function(schema)
    extraction_prompt = prompt or ChatPromptTemplate.from_template(_EXTRACTION_TEMPLATE)
    output_parser = JsonKeyOutputFunctionsParser(key_name="info")
    llm_kwargs = get_llm_kwargs(function)
    custom_chain = CustomExtractionChain(
        llm=llm,
        prompt=extraction_prompt,
        llm_kwargs=llm_kwargs,
        output_parser=output_parser,
        tags=tags,
        verbose=verbose,
    )
    return custom_chain

시퀀셜 체인내에 분기 + 추출 하는 체인을 만듭니다.

In [13]:
memory_key = "foo"
input_key = "input"
output_key = "out_text"

# Initialize the context with a prompt template
memory_template = r"""The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. If the 
AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{}
Human: {}
AI:""".format("{" + memory_key + "}", "{" + input_key + "}")

prompt = PromptTemplate(
    input_variables=[memory_key, input_key], template=memory_template
)

# Initialize memory to store conversation history
memory = ConversationBufferMemory(
    memory_key=memory_key, input_key=input_key, output_key=output_key
)

# Initialize and return conversation chain
con_chain = ConversationChain(
    llm=llm, memory=memory, prompt=prompt, verbose=True,
    input_key=input_key, output_key=output_key
)

In [14]:
# 분기 + 추출 테스트 + 시퀀셜
from langchain.chains import SequentialChain
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
PATH = "./chain_prompts"

ABLE = os.path.join(
    PATH, "ability.txt"
)
MENTOR = os.path.join(
    PATH, "mentor.txt"
)
INTRO = os.path.join(
    PATH, "intro.txt"
)

schema = {
    "properties": {
        "cutomer_name": {"type": "string"},
        "cutomer_feeling": {"type": "string"},
        "cutomer_age": {"type": "string"},
        "cutomer_extra_info": {"type": "string"},
    },
    "required": ["이름"],
}
tag_schema  = {
    "properties": {
        "sentiment": {"type": "string", "enum": ["행복", "중립", "슬픔", "분노", "불안", "기쁨", "불안", "놀람", "기대", "기타"]},
        "problem": {"type": "string", "enum": ["가족", "애인", "직장", "학교", "기타"]},
    }
}

def read_prompt_template(file_path: str) -> str:
    with open(file_path, "r", encoding='UTF8') as f:
        prompt_template = f.read()

    return prompt_template

def create_chain(llm, template_path, output_key):
    return LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template(
            template=read_prompt_template(template_path)
        ),
        output_key=output_key,
        verbose=True,
    )

able = create_chain(
    llm=llm,
    template_path=ABLE,
    output_key="out_text",
)
mentor = create_chain(
    llm=llm,
    template_path=MENTOR,
    output_key="out_text",
)
intro = create_chain(
    llm=llm,
    template_path=INTRO,
    output_key="out_text",
)

tag_chain = create_tagging_chain(tag_schema, llm)
extract_chain = create_custom_extraction_chain(schema, llm)

destinations = [
    # "능력: 챗봇이 할수있는 것들을 알려줍니다",
    "Mentor: 추천할 상담사 목록",
    "building: This is where you'll find the rules for buildings as you play the board game.",
    "intro_AI: This is where you'll find the introduction of myself and my abilities.",
    # "고객에대해: 고객이 자신에 대해서 말할때.",
]
destinations = "\n".join(destinations)
router_prompt_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations)
router_prompt = PromptTemplate.from_template(
    template=router_prompt_template, output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt, verbose=True, output_key="out_text")

multi_prompt_chain = CustomPromptChain( # 멀티프롬프트체인 라우터체인과 데이터네이션 체인 그리고 디펄트체의 합체
    router_chain=router_chain, # 라우터를 쓰면서 텍스트양이 반으로 줄어듬 정확도도 올라감

    destination_chains={
    "Mentor": mentor,
    "intro_AI": intro, # 소개를 질문받으면 답변합니다.
},
    default_chain=con_chain # 디펄트 체인을 설정합니다.
)

overall = SequentialChain(
    chains=[
        multi_prompt_chain,
        extract_chain,
        tag_chain,
    ],
    input_variables=["input"],
    output_variables=["out_text","extracted_data", "text"],
)

class UserRequest(BaseModel):
    user_message: str

def gernerate_answer(req: UserRequest) -> Dict[str, str]:
    context = req.dict()
    context["input"] = context["user_message"]
    answer = overall.invoke(context)
    # answer = multi_prompt_chain.run(context)

    return {"answer": answer}

# 문제가 발생하면 상담사 추천하는 로직을 구현

### User 데이터 입력
* 유저 데이터 입력 후 결과를 확인 합니다.

In [22]:
user_data = {
    # "user_message":"저는 15살 이미려입니다. 마음이 슬퍼요. 상담을 받고싶어요",
    "user_message":"저는 30살 이명지입니다. 가족문제로 마음이 슬퍼요. 상담을 받고싶습니다..",
    # "user_message":"너무 좋은데? 너는 누구니?",
    # "user_message":" 너는 누구니"
    # "user_message":"여자친구와 싸웠어. 어떻게 해야할까?",
    # "user_message":"아들의 학교생활이 걱정돼요. 어떻게 해야할까요?",
    # "user_message":"내 이름은 김지한입니다. 30살이고요.",
    # "user_message":"오늘날씨가 어때요? 제 이름 기억하시나요?",
}
request_instance = UserRequest(**user_data)
gernerate_answer(request_instance)



> Entering new LLMRouterChain chain...


F:\Anaconda3_envs\tensor2\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: [persona]
1. 말을 거는 사람은 고객이며, A.I. 항상 고객의 감정을 위로해주거나 공감하는 말을 추천하기전에 우선 합니다.
2. 20세 이상의 성인에게는 청소년상담사 자격증을 지닌 상담사 추천하지 않습니다.(이 내용을 출력하지 않습니다.)
3. 20세 미만의 경우에는 청소년 상담사를 추천합니다.(이 내용을 출력하지 않습니다.)
4. 애인과의 다툼등은 연애문제로 분류하고 연애문제가 주요분야인 상담사를 추천합니다.(이 내용을 출력하지 않습니다.)
5. 진로상담은 학생에게만 추천합니다.(이 내용을 출력하지 않습니다.)

[상담사목록]

나미선
별칭: 해피매직
대표문구:행복의 기적을 찾아갑니다.
자격증: 상담심리사1급
연차: 15년
주요분야: 부부문제, 이별문제, 자존감, 우울, 불안, 스트레스, 인생고민, 직장문제, 대인관계, 가족문제, 자녀교육, 진로상담,

박진주
별칭: 소울힐러
대표문구: 마음의 H.P를 채워드립니다.
자격증: 상담심리사2급
연차 :3년
주요분야: 연애문제, 이별문제, 재취업

이다함
별칭: 라이프 트레이너
대표문구: 험란한 인생게임을 즐겁게
자격증: 청소년 상담사2급
연차: 5년
주요문제: 진로상담, 우울, 불안, 스트레스,학생 재능발굴

[message]
저는 30살 이명지입니다. 가족문제로 마음이 슬퍼요. 상담을 받고싶습니다..

Answer:

> Finished chain.


{'answer': {'user_message': '저는 30살 이명지입니다. 가족문제로 마음이 슬퍼요. 상담을 받고싶습니다..',
  'input': '저는 30살 이명지입니다. 가족문제로 마음이 슬퍼요. 상담을 받고싶습니다..',
  'out_text': '안녕하세요, 이명지님. 가족문제로 마음이 슬픈 것 같아요. 상담을 받아보시는 것이 좋을 것 같습니다. 저희 상담사 중에서는 부부문제, 이별문제, 자존감, 우울, 불안, 스트레스, 인생고민, 직장문제, 대인관계, 가족문제, 자녀교육에 대한 상담을 주요 분야로 하는 나미선 상담사를 추천해드릴 수 있어요. 어떠신가요?',
  'extracted_data': [{'cutomer_name': '이명지',
    'cutomer_feeling': '슬퍼요',
    'cutomer_age': '30살'}],
  'text': {'problem': '가족', 'sentiment': '슬픔'}}}

## 현재 고쳐야 할것
분기시 대화가 이어지지 않음
모든 대화가 기억하고 이어져야함
추출된 정보를 저장하고 이용해야함
추출된 정보를 이용해 상담사 추천해야함
추출된 정보를 이용해 대화를 이어나가야함

### 구현해야 하는 내용
1.현재 대화내용 기억 
2.과거 대화내용 저장 및 기억
3.웹서비스 하에서 대화표현 (한글자씩 나오게 하기)
4. 만약에 문제가 발견되면 문제에 따른 상담사 추천하는 로직을 구현
5. 이모티콘 활용
### 구현 된 내용
- 멀티프롬프트 체인을 이용한 분기처리
- 추출체인을 이용한 정보추출
- 태깅체인을 이용한 태깅처리
- 시퀀셜체인을 이용한 체인연결